### RESEARCH QUESTION: Yellow vs HVFHV demand

Data to consider: 1.  Yellow Taxis + High Volume FHV
                  2. External data: weather, holidays?

Note: excluding from fhv file cus incomplete data (tips, money, fees etc)

In [4]:
from pyspark.sql import SparkSession

# Create a spark session 
spark = (
    SparkSession.builder.appName("TLC eda")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

### Read Raw data

In [3]:
sdf_yellow_july = spark.read.parquet("../data/tlc_data/raw/yellow/2024-07")
sdf_yellow_july.limit(5)


vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_miles,ratecode_id,store_and_fwd_flag,pu_location_id,do_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2024-07-01 00:34:56,2024-07-01 00:46:49,1,3.2,1,false,140,79,1,15.6,3.5,0.5,3.5,0.0,1.0,24.1,2.5,0.0
2,2024-06-30 23:48:58,2024-07-01 00:28:04,1,19.48,2,false,132,113,2,70.0,0.0,0.5,0.0,0.0,1.0,75.75,2.5,1.75
2,2024-07-01 00:23:18,2024-07-01 00:29:51,1,1.18,1,false,237,145,1,8.6,1.0,0.5,2.72,0.0,1.0,16.32,2.5,0.0
1,2024-07-01 00:10:33,2024-07-01 00:27:31,0,9.1,1,false,138,164,1,36.6,10.25,0.5,12.05,0.0,1.0,60.4,2.5,1.75
1,2024-07-01 00:07:55,2024-07-01 00:34:34,1,17.7,2,false,132,263,1,70.0,1.75,0.5,10.0,6.94,1.0,90.19,0.0,1.75


In [4]:
sdf_yellow_july.count()

3076903

In [5]:
sdf_yellow_july.printSchema()

root
 |-- vendor_id: integer (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- ratecode_id: integer (nullable = true)
 |-- store_and_fwd_flag: boolean (nullable = true)
 |-- pu_location_id: integer (nullable = true)
 |-- do_location_id: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [9]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, unix_timestamp, round, abs

In [ ]:
# Calculate duration in minutes
sdf_yellow_july = sdf_yellow_july.withColumn("trip_time",round((unix_timestamp("dropoff_datetime") - unix_timestamp("pickup_datetime")) / 60, 2))

In [8]:
# match zones
zones_df = spark.read.csv("../data/taxi_zones/taxi+_zone_lookup.csv", header=True, inferSchema=True)
zones_df.show(10)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 10 rows


In [9]:
# change zone column names to match zones and join

zones_df = zones_df.withColumnRenamed("LocationID", "pu_location_id") \
                   .withColumnRenamed("Zone", "pu_zone") \
                   .withColumnRenamed("Borough", "pu_borough") \
                   .withColumnRenamed("service_zone", "pu_service_zone")
                   
sdf_yellow_july = sdf_yellow_july.join(zones_df, on="pu_location_id", how="left")
sdf_yellow_july.show()

+--------------+---------+-------------------+-------------------+---------------+----------+-----------+------------------+--------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+----------+--------------------+---------------+
|pu_location_id|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_miles|ratecode_id|store_and_fwd_flag|do_location_id|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|trip_time|pu_borough|             pu_zone|pu_service_zone|
+--------------+---------+-------------------+-------------------+---------------+----------+-----------+------------------+--------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+----------+--------------------+---------------+
|           1

In [10]:
# again, but for drop off
zones_df = zones_df.withColumnRenamed("pu_location_id", "do_location_id") \
                   .withColumnRenamed("pu_zone", "do_zone") \
                   .withColumnRenamed("pu_borough", "do_borough")\
                   .withColumnRenamed("pu_service_zone", "do_service_zone")

In [11]:
sdf_yellow_july = sdf_yellow_july.join(zones_df, on="do_location_id", how="left")
sdf_yellow_july.show(10)

+--------------+--------------+---------+-------------------+-------------------+---------------+----------+-----------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+----------+--------------------+---------------+----------+--------------------+---------------+
|do_location_id|pu_location_id|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_miles|ratecode_id|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|trip_time|pu_borough|             pu_zone|pu_service_zone|do_borough|             do_zone|do_service_zone|
+--------------+--------------+---------+-------------------+-------------------+---------------+----------+-----------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+---------

25/08/17 20:05:05 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [12]:
sdf_yellow_july = sdf_yellow_july.drop("pu_service_zone")
sdf_yellow_july = sdf_yellow_july.drop("do_service_zone")
sdf_yellow_july.show(10)

+--------------+--------------+---------+-------------------+-------------------+---------------+----------+-----------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+----------+--------------------+----------+--------------------+
|do_location_id|pu_location_id|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_miles|ratecode_id|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|trip_time|pu_borough|             pu_zone|do_borough|             do_zone|
+--------------+--------------+---------+-------------------+-------------------+---------------+----------+-----------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+----------+-------------------

### Check trip distance

In [13]:

sdf_yellow_july.select(F.min("trip_miles").alias("min_distance"),F.max("trip_miles").alias("max_distance")).show()

+------------+------------+
|min_distance|max_distance|
+------------+------------+
|         0.0|   326505.45|
+------------+------------+



In [14]:
sdf_yellow_july = sdf_yellow_july.filter((F.col("trip_miles") > 0.0) & (F.col("trip_miles") <= 200.0))
sdf_yellow_july.count()

3028762

In [15]:
sdf_yellow_july.select("trip_miles", "trip_time", "fare_amount",).filter("trip_time > 1000").show(100, truncate=False)

+----------+---------+-----------+
|trip_miles|trip_time|fare_amount|
+----------+---------+-----------+
|10.31     |1386.13  |44.3       |
|17.69     |1417.15  |70.0       |
|2.22      |1429.23  |13.5       |
|1.72      |1410.75  |10.0       |
|8.79      |1333.72  |35.9       |
|3.7       |1384.88  |17.7       |
|1.98      |1412.95  |12.1       |
|0.94      |1423.68  |7.9        |
|1.71      |1389.13  |10.7       |
|2.01      |1436.3   |10.7       |
|3.79      |1399.53  |18.4       |
|1.54      |1430.1   |10.0       |
|1.1       |1399.75  |17.0       |
|13.35     |1414.68  |72.3       |
|1.39      |1399.57  |12.1       |
|2.18      |1408.23  |17.7       |
|2.21      |1396.08  |18.4       |
|1.45      |1386.8   |9.3        |
|1.79      |1416.62  |14.2       |
|3.2       |1400.1   |20.5       |
|1.23      |1388.77  |10.0       |
|4.08      |1421.53  |26.1       |
|0.8       |1436.63  |8.6        |
|3.32      |1416.68  |19.1       |
|0.69      |1424.83  |6.5        |
|2.65      |1426.43 

### Trip Duration

In [16]:
sdf_yellow_july.select(
    F.min("trip_time").alias("min_duration"),
    F.max("trip_time").alias("max_duration")
).show()

+------------+------------+
|min_duration|max_duration|
+------------+------------+
|       -40.5|     9670.28|
+------------+------------+



In [17]:
sdf_yellow_july = sdf_yellow_july.filter((F.col("trip_time") > 0.0 ) & (F.col("trip_time") < 300.0))
sdf_yellow_july.count()

3026920

In [18]:
# Calculate speed to identify potential sus trip duration
sdf_yellow_july = sdf_yellow_july.withColumn("avg_speed_mph", round(F.col("trip_miles") / (F.col("trip_time") / 60),2))
sdf_yellow_july.show()

+--------------+--------------+---------+-------------------+-------------------+---------------+----------+-----------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+----------+--------------------+----------+--------------------+-------------+
|do_location_id|pu_location_id|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_miles|ratecode_id|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|trip_time|pu_borough|             pu_zone|do_borough|             do_zone|avg_speed_mph|
+--------------+--------------+---------+-------------------+-------------------+---------------+----------+-----------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+--

In [19]:
sdf_yellow_july.select("trip_miles", "trip_time", "fare_amount", "avg_speed_mph", "congestion_surcharge").filter("trip_time > 70").show(100, truncate=False)

+----------+---------+-----------+-------------+--------------------+
|trip_miles|trip_time|fare_amount|avg_speed_mph|congestion_surcharge|
+----------+---------+-----------+-------------+--------------------+
|64.18     |94.47    |411.8      |40.76        |2.5                 |
|40.1      |71.63    |143.7      |33.59        |0.0                 |
|59.87     |80.07    |352.3      |44.86        |0.0                 |
|53.31     |78.48    |264.8      |40.76        |0.0                 |
|38.0      |70.47    |70.0       |32.35        |0.0                 |
|23.62     |187.13   |-177.3     |7.57         |-2.5                |
|23.62     |187.13   |177.3      |7.57         |2.5                 |
|12.1      |125.48   |49.5       |5.79         |0.0                 |
|45.67     |70.18    |189.6      |39.05        |0.0                 |
|14.2      |106.35   |52.5       |8.01         |0.0                 |
|13.0      |72.78    |51.5       |10.72        |0.0                 |
|18.8      |82.17   

In [20]:
sdf_yellow_july = sdf_yellow_july.filter((F.col("avg_speed_mph") >= 3) & (F.col("avg_speed_mph") < 80))
sdf_yellow_july.count()

2979029

In [21]:
sdf_yellow_july.select("trip_miles", "trip_time", "fare_amount", "avg_speed_mph").filter("avg_speed_mph < 12").show(100,truncate=False )

+----------+---------+-----------+-------------+
|trip_miles|trip_time|fare_amount|avg_speed_mph|
+----------+---------+-----------+-------------+
|1.18      |6.55     |8.6        |10.81        |
|2.21      |12.52    |14.9       |10.59        |
|0.9       |5.43     |7.2        |9.94         |
|1.0       |5.58     |7.9        |10.75        |
|0.8       |6.93     |6.5        |6.93         |
|0.57      |2.87     |5.1        |11.92        |
|3.3       |21.33    |20.5       |9.28         |
|6.2       |33.33    |35.2       |11.16        |
|0.77      |4.82     |6.5        |9.59         |
|1.34      |8.43     |9.3        |9.54         |
|2.51      |13.53    |15.6       |11.13        |
|2.9       |17.08    |19.1       |10.19        |
|1.33      |9.38     |10.7       |8.51         |
|0.36      |2.15     |4.4        |10.05        |
|0.49      |2.67     |5.1        |11.01        |
|0.88      |8.4      |8.6        |6.29         |
|3.91      |20.45    |24.0       |11.47        |
|0.95      |6.33    

# Voided trips

kiv on disputed trips

In [22]:
sdf_yellow_july.select("trip_miles", "trip_time", "fare_amount", "avg_speed_mph").filter("payment_type == 6").count()

0

In [23]:
sdf_yellow_july = sdf_yellow_july.filter(F.col("payment_type") != 6)
sdf_yellow_july.count()

2979029

### Fare amount

In [24]:
sdf_yellow_july.select(
    F.min("fare_amount").alias("min_fare"),
    F.max("fare_amount").alias("max_fare")
).show()

+--------+--------+
|min_fare|max_fare|
+--------+--------+
|  -940.0|  1111.1|
+--------+--------+



In [25]:
sdf_yellow_july = sdf_yellow_july.filter(F.col("fare_amount") > 3 )
sdf_yellow_july.count()

2917475

In [26]:
sdf_yellow_july.filter("fare_amount > 200").show(100)


+--------------+--------------+---------+-------------------+-------------------+---------------+----------+-----------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+----------+--------------------+----------+--------------+-------------+
|do_location_id|pu_location_id|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_miles|ratecode_id|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|trip_time|pu_borough|             pu_zone|do_borough|       do_zone|avg_speed_mph|
+--------------+--------------+---------+-------------------+-------------------+---------------+----------+-----------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+----------+---

In [28]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
import builtins

def estimate_fare(trip_distance, trip_duration_min, ratecode_id, avg_speed):
    base_fare = 3.00
    jfk_manhattan = 70

    if avg_speed >= 12:
        distance_fare = 0.70 * (trip_distance / 0.2)
        waiting_fare = 0.0
    else:
        distance_fare = 0.0
        waiting_fare = 0.70 * trip_duration_min
            
    standard_fare = base_fare + distance_fare + waiting_fare
    jfk_manhattan = 70.0
    
    # Out-of-city rate (Nassau/Whechester)
    if ratecode_id == 4:  
        estimated_fare = standard_fare * 1.5  
    # Standard city rate
    elif ratecode_id == 1:  
        estimated_fare = standard_fare
    # JFK Manhattan rate
    elif ratecode_id == 2:
        estimated_fare = jfk_manhattan
    # EWR
    elif ratecode_id == 3:
        estimated_fare = standard_fare + 20
    else:
        estimated_fare = None
        
    if estimated_fare is not None:
        return builtins.round(estimated_fare, 2)
    else:
        return None

estimate_fare_udf = udf(estimate_fare, DoubleType())

In [29]:
sdf_yellow_july = sdf_yellow_july.withColumn(
    "estimated_fare",
    estimate_fare_udf(
        sdf_yellow_july["trip_miles"],
        sdf_yellow_july["trip_time"],
        sdf_yellow_july["ratecode_id"],
        sdf_yellow_july["avg_speed_mph"]
    )
)

In [30]:
sdf_yellow_july.select("trip_miles","pu_zone","do_zone", "fare_amount", "estimated_fare").filter((F.col("ratecode_id") == 4) & (F.col("fare_amount") > 500)).show(100,truncate=False )

+----------+----------------------------+-----------------+-----------+--------------+
|trip_miles|pu_zone                     |do_zone          |fare_amount|estimated_fare|
+----------+----------------------------+-----------------+-----------+--------------+
|93.03     |Midtown Center              |NA               |608.5      |492.91        |
|107.97    |JFK Airport                 |NA               |653.3      |571.34        |
|87.6      |JFK Airport                 |NA               |601.5      |464.4         |
|96.2      |JFK Airport                 |NA               |675.0      |509.55        |
|161.25    |East Village                |LaGuardia Airport|1097.1     |851.06        |
|94.74     |JFK Airport                 |NA               |647.0      |501.88        |
|91.25     |LaGuardia Airport           |NA               |537.1      |483.56        |
|92.18     |JFK Airport                 |NA               |565.8      |488.44        |
|84.15     |JFK Airport                 |NA

In [31]:
sdf_yellow_july = sdf_yellow_july.withColumn(
    "fare_diff",
    F.abs(F.col("fare_amount") - F.col("estimated_fare"))
)

In [32]:
# Filter rows where absolute difference is 100
sdf_yellow_july = sdf_yellow_july.filter((F.col("estimated_fare").isNotNull()) & (F.col("fare_diff") <= 100.0))
sdf_yellow_july.count()

2612403

### passenger count

In [33]:
sdf_yellow_july.filter(F.col("passenger_count") == 0).count()

26999

In [34]:
sdf_yellow_july = sdf_yellow_july.filter((F.col("passenger_count") >= 1) & (F.col("passenger_count") < 6))
sdf_yellow_july.count()

2570406

### Check if total amount aligns with everything

*** kiv on this

In [35]:
# Add a calculated total
sdf_yellow_july = sdf_yellow_july.withColumn(
    "calc_total",
    F.col("fare_amount")
    + F.col("extra")
    + F.col("mta_tax")
    + F.col("tip_amount")
    + F.col("tolls_amount")
    + F.col("improvement_surcharge")
    + F.col("congestion_surcharge")
    + F.col("airport_fee")
)

In [36]:
sdf_yellow_july.filter(F.round(F.abs(F.col("total_amount") - F.col("calc_total"))) > 4.0).count()

0

In [37]:
sdf_yellow_july.limit(5)

do_location_id,pu_location_id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_miles,ratecode_id,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,trip_time,pu_borough,pu_zone,do_borough,do_zone,avg_speed_mph,estimated_fare,fare_diff,calc_total
79,140,1,2024-07-01 00:34:56,2024-07-01 00:46:49,1,3.2,1,false,1,15.6,3.5,0.5,3.5,0.0,1.0,24.1,2.5,0.0,11.88,Manhattan,Lenox Hill East,Manhattan,East Village,16.16,14.2,1.4000000000000004,26.6
113,132,2,2024-06-30 23:48:58,2024-07-01 00:28:04,1,19.48,2,false,2,70.0,0.0,0.5,0.0,0.0,1.0,75.75,2.5,1.75,39.1,Queens,JFK Airport,Manhattan,Greenwich Village...,29.89,70.0,0.0,75.75
145,237,2,2024-07-01 00:23:18,2024-07-01 00:29:51,1,1.18,1,false,1,8.6,1.0,0.5,2.72,0.0,1.0,16.32,2.5,0.0,6.55,Manhattan,Upper East Side S...,Queens,Long Island City/...,10.81,7.58,1.0199999999999996,16.32
263,132,1,2024-07-01 00:07:55,2024-07-01 00:34:34,1,17.7,2,false,1,70.0,1.75,0.5,10.0,6.94,1.0,90.19,0.0,1.75,26.65,Queens,JFK Airport,Manhattan,Yorkville West,39.85,70.0,0.0,91.94
170,114,2,2024-07-01 00:19:42,2024-07-01 00:32:13,1,2.21,1,false,2,14.9,1.0,0.5,0.0,0.0,1.0,19.9,2.5,0.0,12.52,Manhattan,Greenwich Village...,Manhattan,Murray Hill,10.59,11.76,3.1400000000000006,19.9


In [38]:
sdf_yellow_july = sdf_yellow_july.drop("calc_total")

### remove entries with unknown zones

In [39]:
sdf_yellow_july.select("do_zone").distinct().show(300)


+--------------------+
|             do_zone|
+--------------------+
|           Homecrest|
|              Corona|
|    Bensonhurst West|
|         Westerleigh|
|          Douglaston|
|      Newark Airport|
|Charleston/Totten...|
|East Concourse/Co...|
|          Mount Hope|
|      Pelham Parkway|
|         Marble Hill|
|           Rego Park|
|Upper East Side S...|
|       Dyker Heights|
|Heartland Village...|
|   Kew Gardens Hills|
|     Jackson Heights|
|             Bayside|
|      Yorkville West|
|TriBeCa/Civic Center|
|          Highbridge|
|Stuy Town/Peter C...|
|Flushing Meadows-...|
|      Bushwick South|
|Riverdale/North R...|
|     Windsor Terrace|
|         Old Astoria|
|          Ocean Hill|
|        Clinton Hill|
|      West Concourse|
|  Stuyvesant Heights|
|       Broad Channel|
|Upper West Side N...|
|     Mariners Harbor|
|   East Harlem South|
|Springfield Garde...|
|          Kensington|
|     Oakland Gardens|
| Crown Heights North|
|            Longwood|
|     Lenox

In [40]:
sdf_yellow_july.filter((F.col("do_zone") == 'NV') | (F.col("do_zone") == 'NA')  | (F.col("do_zone").isNull())).count()

22651

In [41]:
sdf_yellow_july = sdf_yellow_july.filter(
    (F.col("pu_zone").isNotNull()) &
    (~F.col("pu_zone").isin("NV", "NA")) &
    (F.col("do_zone").isNotNull()) &
    (~F.col("do_zone").isin("NV", "NA"))
)

In [42]:
sdf_yellow_july.count()

2543698

In [43]:
sdf_yellow_july.select("payment_type").distinct().show(300)


+------------+
|payment_type|
+------------+
|           1|
|           3|
|           4|
|           2|
+------------+



## Apply filteration to other yellow dataset months

In [44]:
from pyspark.sql import functions as F
from pyspark.sql.functions import unix_timestamp, round

def clean_yellow(input_path, zones_df):
    # Read parquet file
    df = spark.read.parquet(input_path)
    
    # Calculate trip time in minutes, rounded to 2 decimals
    df = df.withColumn("trip_time", round((unix_timestamp("dropoff_datetime") - unix_timestamp("pickup_datetime")) / 60, 2))
    
    # Map pickup zones
    pu_zones = zones_df.select(
        F.col("LocationID").alias("pu_location_id"),
        F.col("Zone").alias("pu_zone"),
        F.col("Borough").alias("pu_borough"),
        F.col("service_zone").alias("pu_service_zone")
    )
    df = df.join(pu_zones, on="pu_location_id", how="left")
    
    # Map drop-off zones (rename columns for joining)
    do_zones = zones_df.select(
        F.col("LocationID").alias("do_location_id"),
        F.col("Zone").alias("do_zone"),
        F.col("Borough").alias("do_borough"),
        F.col("service_zone").alias("do_service_zone")
    )
    df = df.join(do_zones, on="do_location_id", how="left")
    
    # Drop service zones columns if not needed
    df = df.drop("pu_service_zone", "do_service_zone")
    
    # Filter trip distance outliers
    df = df.filter((F.col("trip_miles") > 0.0) & (F.col("trip_miles") <= 200.0))
    
    # Filter trip time outliers
    df = df.filter((F.col("trip_time") > 0.0) & (F.col("trip_time") < 300.0))
    
    # Calculate average speed mph
    df = df.withColumn("avg_speed_mph", round(F.col("trip_miles") / (F.col("trip_time") / 60), 2))
    
    # Filter unreasonable speeds
    df = df.filter((F.col("avg_speed_mph") >= 3) & (F.col("avg_speed_mph") < 80))
    
    # Filter voided trips
    df = df.filter(F.col("payment_type") != 6)
    
    # Filter low fare amount
    df = df.filter(F.col("fare_amount") > 3)
    
    # Calculate estimated fare (assuming you have defined estimate_fare_udf)
    df = df.withColumn("estimated_fare", estimate_fare_udf(F.col("trip_miles"), F.col("trip_time"), F.col("ratecode_id"), F.col("avg_speed_mph")))
    
    # Calculate absolute fare difference
    df = df.withColumn("fare_diff", F.abs(F.col("fare_amount") - F.col("estimated_fare")))
    
    # Filter where estimated fare exists and difference <= 100
    df = df.filter((F.col("estimated_fare").isNotNull()) & (F.col("fare_diff") <= 100.0))
    
    # Filter passenger count
    df = df.filter((F.col("passenger_count") >= 1) & (F.col("passenger_count") < 6))
    
    # Filter valid pickup and dropoff zones
    df = df.filter(
        (F.col("pu_zone").isNotNull()) &
        (~F.col("pu_zone").isin("NV", "NA")) &
        (F.col("do_zone").isNotNull()) &
        (~F.col("do_zone").isin("NV", "NA"))
    )
    
    df.coalesce(1).write.mode('overwrite').parquet(f'../data/tlc_data/raw/cleaned/yellow/2024-{str(month).zfill(2)}')
    
    return None


In [45]:
zones_df = spark.read.csv("../data/taxi_zones/taxi+_zone_lookup.csv", header=True, inferSchema=True)
for month in range(7, 13):
    input_path = f'../data/tlc_data/raw/yellow/{2024}-{str(month).zfill(2)}'
    clean_yellow(input_path, zones_df)

In [46]:
def qc_check(df):
    # 1. Check for invalid PU/DO zones
    bad_zones_count = df.filter(
        (F.col("pu_zone").isNull()) |
        (F.col("pu_zone").isin("NV", "NA")) |
        (F.col("do_zone").isNull()) |
        (F.col("do_zone").isin("NV", "NA"))
    ).count()
    
    # 2. Check for negative or zero fares
    bad_fares_count = df.filter(F.col("fare_amount") <= 3.0).count()
    
    # 3. Check for unrealistic trip distances
    bad_distance_count = df.filter((F.col("trip_miles") <= 0.0) | (F.col("trip_miles") > 200)).count()
    
    bad_duration_count = df.filter((F.col("trip_time") < 0.0) & (F.col("trip_time") > 300.0)).count()
    
    
    
    print("🚦 QC Results 🚦")
    print(f"Invalid zones remaining: {bad_zones_count}")
    print(f"Bad fares: {bad_fares_count}")
    print(f"Bad distances: {bad_distance_count}")
    print(f"Bad durations: {bad_duration_count}")



In [47]:
cleaned_example = spark.read.parquet("/Users/tasneemzulaiqa/Documents/GitHub/project-1-individual-tasneemzulaiqa/data/tlc_data/raw/cleaned/yellow/2024-12")

In [48]:
qc_check(cleaned_example)

🚦 QC Results 🚦
Invalid zones remaining: 0
Bad fares: 0
Bad distances: 0
Bad durations: 0


In [49]:
cleaned_example.show(100)

+--------------+--------------+---------+-------------------+-------------------+---------------+----------+-----------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+--------------------+----------+--------------------+----------+-------------+--------------+--------------------+
|do_location_id|pu_location_id|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_miles|ratecode_id|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|trip_time|             pu_zone|pu_borough|             do_zone|do_borough|avg_speed_mph|estimated_fare|           fare_diff|
+--------------+--------------+---------+-------------------+-------------------+---------------+----------+-----------+------------------+------------+-----------+-----+-------+----------+------------+--------

In [15]:
all_dfs = []

# Read all months and add 'month' column
for month in range(7, 13):
    month_name = f'2024-{str(month).zfill(2)}'
    df = spark.read.parquet(
        f"/Users/tasneemzulaiqa/Documents/GitHub/project-1-individual-tasneemzulaiqa/data/tlc_data/raw/cleaned/yellow/{month_name}"
    )
    df = df.withColumn("month", F.lit(month_name))
    all_dfs.append(df)

# Union all months into a single DataFrame
yellow_all = all_dfs[0]
for d in all_dfs[1:]:
    yellow_all = yellow_all.unionByName(d)


In [16]:
yellow_all.count()

16945833

### HVFHV

In [11]:
sdf_fhv_example = spark.read.parquet('../data/tlc_data/raw/hvfhv/2024-07')
sdf_fhv_example.show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+--------------+--------------+----------+---------+-----------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|pu_location_id|do_location_id|trip_miles|trip_time|fare_amount|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+--------------+--------------+----------+---------+-----------+-----+----+---------+--------------------+-----------+----+---------

In [104]:
zones_df = spark.read.csv("../data/taxi_zones/taxi+_zone_lookup.csv", header=True, inferSchema=True)

pu_zones = zones_df.select(
    F.col("LocationID").alias("pu_location_id"),
    F.col("Zone").alias("pu_zone"),
    F.col("Borough").alias("pu_borough"),
    F.col("service_zone").alias("pu_service_zone")
)
sdf_fhv_example= sdf_fhv_example.join(pu_zones, on="pu_location_id", how="left")

# Map drop-off zones (rename columns for joining)
do_zones = zones_df.select(
    F.col("LocationID").alias("do_location_id"),
    F.col("Zone").alias("do_zone"),
    F.col("Borough").alias("do_borough"),
    F.col("service_zone").alias("do_service_zone")
)
sdf_fhv_example = sdf_fhv_example.join(do_zones, on="do_location_id", how="left")

# Drop service zones columns if not needed
sdf_fhv_example = sdf_fhv_example.drop("pu_service_zone", "do_service_zone")

In [105]:
sdf_fhv_example.show()

+--------------+--------------+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+----------+---------+-----------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+--------------------+----------+--------------------+----------+
|do_location_id|pu_location_id|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|trip_miles|trip_time|fare_amount|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|             pu_zone|pu_borough|             do_zone|do_borough|
+--------------+--------------+-----------------+--------------------+--------------------+-------------------+-------------------+-------------

In [106]:
sdf_fhv_example.select(
    F.min("trip_time").alias("min_duration"),
    F.max("trip_time").alias("max_duration")
).show()

+------------+------------+
|min_duration|max_duration|
+------------+------------+
|         0.0|     42478.0|
+------------+------------+



In [88]:
sdf_fhv_example = sdf_fhv_example.withColumn(
    "trip_time", 
    F.round(F.col("trip_time") / 60, 2)
)

In [89]:
sdf_fhv_example = sdf_fhv_example.withColumn("calculated_time",round((unix_timestamp("dropoff_datetime") - unix_timestamp("pickup_datetime")) / 60, 2))

In [90]:
sdf_fhv_example.filter(abs(F.col('trip_time') - F.col('calculated_time')) > 1 ).select("trip_time", 'calculated_time').show(100)

+---------+---------------+
|trip_time|calculated_time|
+---------+---------------+
+---------+---------------+



In [91]:
sdf_fhv_example =  sdf_fhv_example.drop('calculated_time')

In [92]:
sdf_fhv_example.filter(F.col('trip_time') > 500 ).select('trip_miles', 'trip_time').show(100)

+----------+---------+
|trip_miles|trip_time|
+----------+---------+
|     0.521|   523.18|
|    19.865|   519.42|
|   126.869|   568.12|
|     7.427|   589.95|
|    34.585|   547.78|
|    108.72|   707.97|
|     3.907|   550.52|
|    70.186|    543.2|
|    103.88|   707.47|
|     14.46|   639.15|
|    62.424|   530.43|
|   219.218|   552.23|
|     92.66|   503.18|
|     75.52|   542.52|
|    62.532|   500.97|
|     83.02|   539.37|
|     3.185|   514.45|
+----------+---------+



### apply same basic cleaning for fhv

In [107]:
from pyspark.sql import functions as F

def clean_hvfhv(input_path, zones_df):
    # Read parquet file
    df = spark.read.parquet(input_path)
    
    # Map pickup zones
    pu_zones = zones_df.select(
        F.col("LocationID").alias("pu_location_id"),
        F.col("Zone").alias("pu_zone"),
        F.col("Borough").alias("pu_borough"),
        F.col("service_zone").alias("pu_service_zone")
    )
    df = df.join(pu_zones, on="pu_location_id", how="left")
    
    # Map drop-off zones
    do_zones = zones_df.select(
        F.col("LocationID").alias("do_location_id"),
        F.col("Zone").alias("do_zone"),
        F.col("Borough").alias("do_borough"),
        F.col("service_zone").alias("do_service_zone")
    )
    df = df.join(do_zones, on="do_location_id", how="left")
    
    # Drop service zone columns
    df = df.drop("pu_service_zone", "do_service_zone")
    
    # Convert trip_time from seconds to minutes
    df = df.withColumn("trip_time", F.round(F.col("trip_time") / 60, 2))
    
    # Remove nulls / NaNs in numeric columns
    numeric_cols = ["trip_miles", "trip_time", "fare_amount", "driver_pay",
                    "tolls", "tips", "bcf", "sales_tax", "congestion_surcharge", "airport_fee"]
    for col in numeric_cols:
        df = df.filter((F.col(col).isNotNull()) & (~F.isnan(F.col(col))))
    
    # Remove nulls in datetime columns
    datetime_cols = ['request_datetime', 'on_scene_datetime', 'pickup_datetime', 'dropoff_datetime']
    for col in datetime_cols:
        df = df.filter(F.col(col).isNotNull())
    
    # Remove nulls in string/categorical columns
    string_cols = ["pu_zone", "do_zone", "pu_borough", "do_borough",
                   'originating_base_num', 'dispatching_base_num', 'hvfhs_license_num']
    for col in string_cols:
        df = df.filter(F.col(col).isNotNull())
    
    # Filter trip distance outliers
    df = df.filter((F.col("trip_miles") > 0.0) & (F.col("trip_miles") <= 200.0))
    
    # Filter trip time outliers
    df = df.filter((F.col("trip_time") > 0.0) & (F.col("trip_time") < 300.0))
    
    # Calculate average speed (mph)
    df = df.withColumn("avg_speed_mph", F.round(F.col("trip_miles") / (F.col("trip_time") / 60), 2))
    
    # Filter unreasonable speeds
    df = df.filter((F.col("avg_speed_mph") >= 3) & (F.col("avg_speed_mph") < 80))
    
    # Filter low fares and driver pay
    df = df.filter((F.col("fare_amount") > 3.0) & (F.col("driver_pay") > 0.0))
    
    # Filter invalid pickup and dropoff zones
    df = df.filter(
        (~F.col("pu_zone").isin("NV", "NA")) &
        (~F.col("do_zone").isin("NV", "NA"))
    )
    
    # Save cleaned data
    df.coalesce(1).write.mode("overwrite").parquet(f'../data/tlc_data/raw/cleaned/hvfhv/2024-{str(month).zfill(2)}')
    
    return None


In [108]:
zones_df = spark.read.csv("../data/taxi_zones/taxi+_zone_lookup.csv", header=True, inferSchema=True)
for month in range(7, 13):
    input_path = f'../data/tlc_data/raw/hvfhv/{2024}-{str(month).zfill(2)}'
    clean_hvfhv(input_path, zones_df)

In [109]:
def qc_check(df):
    # 1. Check for invalid PU/DO zones
    bad_zones_count = df.filter(
        (F.col("pu_zone").isNull()) |
        (F.col("pu_zone").isin("NV", "NA")) |
        (F.col("do_zone").isNull()) |
        (F.col("do_zone").isin("NV", "NA"))
    ).count()
    
    # 2. Check for negative or zero fares
    bad_fares_count = df.filter(F.col("fare_amount") <= 3.0).count()
    
    # 3. Check for unrealistic trip distances
    bad_distance_count = df.filter((F.col("trip_miles") <= 0.0) | (F.col("trip_miles") > 200)).count()
    
    bad_duration_count = df.filter((F.col("trip_time") < 0.0) & (F.col("trip_time") > 300.0)).count()
    
    bad_speed = df.filter((F.col("avg_speed_mph") <= 3) & (F.col("avg_speed_mph") > 80)).count()
    
    
    print("🚦 QC Results 🚦")
    print(f"Invalid zones remaining: {bad_zones_count}")
    print(f"Bad fares: {bad_fares_count}")
    print(f"Bad distances: {bad_distance_count}")
    print(f"Bad durations: {bad_duration_count}")
    print(f"Bad speed: {bad_speed}")

In [110]:
cleaned_example_fhv = spark.read.parquet("/Users/tasneemzulaiqa/Documents/GitHub/project-1-individual-tasneemzulaiqa/data/tlc_data/raw/cleaned/hvfhv/2024-12")

In [111]:
cleaned_example_fhv.count()

15079159

In [112]:
qc_check(cleaned_example_fhv)

🚦 QC Results 🚦
Invalid zones remaining: 0
Bad fares: 0
Bad distances: 0
Bad durations: 0
Bad speed: 0


In [113]:
cleaned_example_fhv.show(10)

+--------------+--------------+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+----------+---------+-----------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+--------------------+-------------+--------------------+-------------+-------------+
|do_location_id|pu_location_id|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|trip_miles|trip_time|fare_amount|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|             pu_zone|   pu_borough|             do_zone|   do_borough|avg_speed_mph|
+--------------+--------------+-----------------+--------------------+--------------------+-------------

In [114]:
for month in range(7, 13):
    path = f'../data/tlc_data/raw/cleaned/hvfhv/2024-{str(month).zfill(2)}'
    df = spark.read.parquet(path)
    count = df.count()
    print(f"Month {month}: {count} rows")

Month 7: 13726156 rows
Month 8: 13508861 rows
Month 9: 13924932 rows
Month 10: 14411755 rows
Month 11: 14407058 rows
Month 12: 15079159 rows


### Aggregating hvfhv

In [3]:
all_dfs = []

# Read all months and add 'month' column
for month in range(7, 13):
    month_name = f'2024-{str(month).zfill(2)}'
    df = spark.read.parquet(
        f"/Users/tasneemzulaiqa/Documents/GitHub/project-1-individual-tasneemzulaiqa/data/tlc_data/raw/cleaned/hvfhv/{month_name}")
    df = df.withColumn("month", F.lit(month_name))
    all_dfs.append(df)

# Union all months into a single DataFrame
hvfhv_all = all_dfs[0]
for d in all_dfs[1:]:
    hvfhv_all = hvfhv_all.unionByName(d)

NameError: name 'F' is not defined

In [5]:
hvfhv_all.show(100)

25/08/17 23:13:52 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------+--------------+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+----------+---------+-----------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+--------------------+-------------+--------------------+-------------+-------------+-------+
|do_location_id|pu_location_id|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|trip_miles|trip_time|fare_amount|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|             pu_zone|   pu_borough|             do_zone|   do_borough|avg_speed_mph|  month|
+--------------+--------------+-----------------+--------------------+------------------

In [6]:
hvfhv_all = hvfhv_all.withColumn("fare_per_mile", F.when(F.col("trip_miles") > 0, F.round(F.col("fare_amount") / F.col("trip_miles"), 2)).otherwise(None))

In [7]:
hvfhv_all = hvfhv_all.withColumn("hour", F.hour("pickup_datetime"))
hvfhv_all = hvfhv_all.withColumn("day_of_week", F.dayofweek("pickup_datetime"))
hvfhv_all = hvfhv_all.withColumn("is_weekend", (F.dayofweek("pickup_datetime").isin([1,7])))


In [8]:
from pyspark.sql import functions as F

hvfhv_all = hvfhv_all.withColumn("total_amount", F.round((F.col("fare_amount") + F.col("tolls") + F.col("bcf") + F.col("sales_tax") + F.col("congestion_surcharge") + F.col("airport_fee") + F.col("tips") + F.col("driver_pay")),2))


In [9]:
hvfhv_all = hvfhv_all.withColumn("tip_rate", F.when(F.col("total_amount") > 0, F.round(F.col("tips") / F.col("total_amount"),2)).otherwise(F.lit(None)))

In [10]:
hvfhv_all.show()

+--------------+--------------+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+----------+---------+-----------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+--------------------+----------+--------------------+----------+-------------+-------+-------------+----+-----------+----------+------------+--------+
|do_location_id|pu_location_id|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|trip_miles|trip_time|fare_amount|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|             pu_zone|pu_borough|             do_zone|do_borough|avg_speed_mph|  month|fare_per_mile|hour|day_of_week|is_wee

In [11]:
hvfhv_all.count()

85057921

In [12]:
hvfhv_all.select("month").distinct()

month
2024-07
2024-08
2024-09
2024-10
2024-11
2024-12


In [13]:
hvfhv_all.printSchema()

root
 |-- do_location_id: integer (nullable = true)
 |-- pu_location_id: integer (nullable = true)
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp_ntz (nullable = true)
 |-- on_scene_datetime: timestamp_ntz (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: boolean (nullable = true)
 |-- shared_match_flag: boolean (nullable = true)
 |-- access_a_

In [28]:
hvfhv_all.select("total_amount", "tips").describe().show()

+-------+------------------+------------------+
|summary|      total_amount|              tips|
+-------+------------------+------------------+
|  count|          85057921|          85057921|
|   mean|50.606120126670824|1.0265195293217706|
| stddev|38.664108556659805| 2.997272871746491|
|    min|              4.87|               0.0|
|    max|           1990.96|             237.1|
+-------+------------------+------------------+



In [7]:
def aggregate_hvfhv(input_path):
    # Read parquet file
    df = spark.read.parquet(input_path)
    
    # Calculate fare per mile
    df = df.withColumn(
        "fare_per_mile",
        F.when(F.col("trip_miles") > 0, F.round(F.col("fare_amount") / F.col("trip_miles"), 2)).otherwise(None)
    )
    
    # Calculate fare per minute
    df = df.withColumn(
        "fare_per_min",
        F.when(F.col("trip_time") > 0, F.round(F.col("fare_amount") / F.col("trip_time"), 2)).otherwise(None)
    )
    
    
    # Calculate total_amount
    df = df.withColumn(
        "total_amount",
        F.round(
            (F.col("fare_amount") + F.col("tolls") + F.col("bcf") + F.col("sales_tax") +
             F.col("congestion_surcharge") + F.col("airport_fee") + F.col("tips") + F.col("driver_pay")), 2)
    )
    
    # Tip rate
    df = df.withColumn(
        "tip_rate",
        F.when(F.col("total_amount") > 0, F.round(F.col("tips") / F.col("total_amount"), 2)).otherwise(None)
    )
    
    # Extract hour, day_of_week, is_weekend
    df = df.withColumn("hour", F.hour("pickup_datetime"))
    df = df.withColumn("day_of_week", F.dayofweek("pickup_datetime"))
    df = df.withColumn("is_weekend", F.col("day_of_week").isin([1,7]))
    
    # Save cleaned data
    df.coalesce(1).write.mode("overwrite").parquet(f'../data/tlc_data/raw/cleaned/curated/hvfhv/2024-{str(month).zfill(2)}')
    
    return None

In [10]:
for month in range(7, 13):
    input_path = f'../data/tlc_data/raw/cleaned/hvfhv/{2024}-{str(month).zfill(2)}'
    aggregate_hvfhv(input_path)

25/08/18 16:08:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [12]:
sdf_fhv_example = spark.read.parquet('../data/tlc_data/raw/cleaned/curated/hvfhv/2024-07')
sdf_fhv_example.show()

+--------------+--------------+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+----------+---------+-----------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+--------------------+----------+--------------------+----------+-------------+-------------+------------+------------+--------+----+-----------+----------+
|do_location_id|pu_location_id|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|trip_miles|trip_time|fare_amount|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|             pu_zone|pu_borough|             do_zone|do_borough|avg_speed_mph|fare_per_mile|fare_per_min|total_amount|

In [ ]:
sdf_fhv_example.printSchema()

root
 |-- do_location_id: integer (nullable = true)
 |-- pu_location_id: integer (nullable = true)
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp_ntz (nullable = true)
 |-- on_scene_datetime: timestamp_ntz (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: boolean (nullable = true)
 |-- shared_match_flag: boolean (nullable = true)
 |-- access_a_

25/08/18 17:11:49 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 215303 ms exceeds timeout 120000 ms
25/08/18 17:11:49 WARN SparkContext: Killing executors is not supported by current scheduler.
25/08/18 17:11:53 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

### Aggregating Yellow dataset

In [17]:
yellow_all.show(100)

+--------------+--------------+---------+-------------------+-------------------+---------------+----------+-----------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+--------------------+----------+--------------------+----------+-------------+--------------+-------------------+-------+
|do_location_id|pu_location_id|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_miles|ratecode_id|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|trip_time|             pu_zone|pu_borough|             do_zone|do_borough|avg_speed_mph|estimated_fare|          fare_diff|  month|
+--------------+--------------+---------+-------------------+-------------------+---------------+----------+-----------+------------------+------------+-----------+-----+-------+----------+-------

In [18]:
yellow_curated = yellow_all.drop("fare_diff", "estimated_fare")

In [19]:
yellow_curated.count()

16945833

In [20]:
yellow_curated = yellow_curated.withColumn("fare_per_mile", F.when(F.col("trip_miles") > 0, F.round(F.col("fare_amount") / F.col("trip_miles"), 2)).otherwise(None))

In [21]:
yellow_curated = yellow_curated.withColumn(
    "fare_per_minute",
    F.when(F.col("trip_time") > 0, F.round(F.col("fare_amount") / (F.col("trip_time")/60), 2))
     .otherwise(None)
)

In [22]:
yellow_curated.printSchema()

root
 |-- do_location_id: integer (nullable = true)
 |-- pu_location_id: integer (nullable = true)
 |-- vendor_id: integer (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- ratecode_id: integer (nullable = true)
 |-- store_and_fwd_flag: boolean (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- trip_time: double (nullable = true)
 |-- pu_zone: string (nullable = true)
 |-- pu_borough: string (nullable = true)
 

In [23]:
yellow_curated = yellow_curated.withColumn("hour", F.hour("pickup_datetime"))
yellow_curated = yellow_curated.withColumn("day_of_week", F.dayofweek("pickup_datetime"))
yellow_curated = yellow_curated.withColumn("is_weekend", (F.dayofweek("pickup_datetime").isin([1,7])))


In [24]:
yellow_curated.show(100)

+--------------+--------------+---------+-------------------+-------------------+---------------+----------+-----------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+--------------------+----------+--------------------+----------+-------------+-------+-------------+---------------+----+-----------+----------+
|do_location_id|pu_location_id|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_miles|ratecode_id|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|trip_time|             pu_zone|pu_borough|             do_zone|do_borough|avg_speed_mph|  month|fare_per_mile|fare_per_minute|hour|day_of_week|is_weekend|
+--------------+--------------+---------+-------------------+-------------------+---------------+----------+-----------+------------------+-----------

In [25]:
yellow_curated.count()

16945833

In [26]:
yellow_curated = yellow_curated.withColumn(
    "tip_rate",
    F.when(F.col("payment_type") == 1,  
           F.when(F.col("total_amount") > 0, round(F.col("tip_amount") / F.col("total_amount"),2))
            .otherwise(None)
    ).otherwise(None)
)


In [27]:
yellow_curated.show()

+--------------+--------------+---------+-------------------+-------------------+---------------+----------+-----------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+--------------------+----------+--------------------+----------+-------------+-------+-------------+---------------+----+-----------+----------+--------+
|do_location_id|pu_location_id|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_miles|ratecode_id|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|trip_time|             pu_zone|pu_borough|             do_zone|do_borough|avg_speed_mph|  month|fare_per_mile|fare_per_minute|hour|day_of_week|is_weekend|tip_rate|
+--------------+--------------+---------+-------------------+-------------------+---------------+----------+-----------+------------

In [28]:
yellow_curated.filter(F.col("payment_type") != 1).show()

+--------------+--------------+---------+-------------------+-------------------+---------------+----------+-----------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+--------------------+----------+--------------------+----------+-------------+-------+-------------+---------------+----+-----------+----------+--------+
|do_location_id|pu_location_id|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_miles|ratecode_id|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|trip_time|             pu_zone|pu_borough|             do_zone|do_borough|avg_speed_mph|  month|fare_per_mile|fare_per_minute|hour|day_of_week|is_weekend|tip_rate|
+--------------+--------------+---------+-------------------+-------------------+---------------+----------+-----------+------------

In [29]:
yellow_curated.count()

16945833

In [30]:
yellow_curated.printSchema()

root
 |-- do_location_id: integer (nullable = true)
 |-- pu_location_id: integer (nullable = true)
 |-- vendor_id: integer (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- ratecode_id: integer (nullable = true)
 |-- store_and_fwd_flag: boolean (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- trip_time: double (nullable = true)
 |-- pu_zone: string (nullable = true)
 |-- pu_borough: string (nullable = true)
 

In [ ]:
output_path = "../data/tlc_data/raw/cleaned/curated/yellow"

In [68]:
months = yellow_curated.select("month").distinct().collect()
for m in months:
    month_val = m["month"]
    yellow_curated.filter(F.col("month") == month_val) \
        .write.mode("overwrite") \
        .parquet(f"{output_path}/{month_val}")
